In [1]:
!nvidia-smi

Thu Aug 24 03:13:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqg einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off



Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -g


In [3]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

In [4]:
questions_dir = Path("skyscanner")
questions_dir.mkdir(exist_ok=True, parents=True)

def write_file(question, answer, file_path):
  text = f"""
Q: {question}
A: {answer}
""".strip()
  with Path(questions_dir / file_path).open("w") as text_file:
    text_file.write(text)


In [5]:
write_file(
    question="How do I search for flights on Skyscanner?",
    answer="""
We know you're looking for the best prices and maximum flexibility to choose the right flight.

To start your search, click here to head back to Skyscanner's search pages. Enter the departure country, city or airport in the From text field, the destination country, city or airport in the To text field and your travel dates in the Depart and Return text fields, and click Search. A list of flights will appear.

When you've seen a suitable flight for your needs, you can click on Select, and we'll show you a list of airlines and travel agencies so you can book directly with them.

Smart search filters, such as number of stops and departure time, help you find the perfect flight. Plus, we've got many tips and tricks to help you save more. Please visit our Travel Blog page to find out more.

If you're looking for inspiration for your next trip, why not try our everywhere feature, which helps you see endless destinations, at every price, from thousands of travel sites in one place?
""".strip(),
    file_path="question_1.txt"
)

In [6]:
write_file(
    question="What are mash-ups?",
    answer="""
These are routes where you fly with different airlines, because it’s cheaper than booking with just one. For example:

If you wanted to fly London to New York, we might find it’s cheaper to fly out with British Airways and back with Virgin Atlantic, rather than buy a return ticket with one airline. This is called a ‘sum-of-one-way’ mash-up. Just in case you’re interested.

Another kind of mash-up is what we call a ‘self-transfer’ or a ‘non-protected transfer’. For example:

If you wanted to fly London to Sydney, we might find it’s cheaper to fly London to Dubai with Emirates, and then Dubai to Sydney with Qantas, rather than booking the whole route with one airline.

Pretty simple, right?

However, what’s really important to bear in mind is that mash-ups are NOT codeshares. A codeshare is when the airlines have an alliance. If anything goes wrong with the route — a delay, say, or a strike — those airlines will help you out at no extra cost. But mash-ups DO NOT involve an airline alliance. So if something goes wrong with a mash-up, it could cost you more money.
""".strip(),
    file_path="question_2.txt"
)

In [7]:
write_file(
    question="Why have I been blocked from accessing the Skyscanner website?",
    answer="""
Skyscanner’s websites are scraped by bots many millions of times a day which has a detrimental effect on the service we’re able to provide. To prevent this, we use a bot blocking solution which checks to ensure you’re using the website in a normal manner.

Occasionally, this may mean that a genuine user may be wrongly flagged as a bot. This can be for a number of potential reasons, including, but not limited to:

You’re using a VPN which we have had to block due to excessive bot traffic in the past
You’re using our website at super speed which manages to beat our rate limits
You have a plug-in on your browser which could be interfering with how our website interacts with you as a user
You’re using an automated browser
If you've been blocked during normal use, please send us your IP address (this website may help: http://www.whatismyip.com/), the website you’re accessing (e.g. www.skyscanner.net) and the date/time this happened, via the Contact Us button below and we’ll look into it as quickly as possible.
""".strip(),
    file_path="question_3.txt"
)

In [8]:
write_file(
    question="Where is my booking confirmation?",
    answer="""
You should get a booking confirmation by email from the company you bought your travel from. This can sometimes go into your spam/junk mail folder so it's always worth checking there.

If you still can't find it, it's worth getting in touch with the company you bought from to find out what's going on.

To find out who you need to contact, check the company name next to the charge on your bank account.
""".strip(),
    file_path="question_4.txt"
)

In [9]:
write_file(
    question="How do I change or cancel my booking?",
    answer="""
For all questions about changes, cancellations, and refunds – as well as all other questions about bookings – you'll need to contact the company you bought travel from. They'll have all the info about your booking and can advise you.

You'll find 1000s of travel agencies, airlines, hotels and car hire companies that you can buy from through our site and app. When you buy from one of these travel partners, they will take your payment (you'll see their name on your bank or credit card statement), contact you to confirm your booking, and provide any help or support you might need.

If you bought from one of these partners, you'll need to contact them as they have all the info about your booking. We unfortunately don't have any access to bookings you made with them.
""".strip(),
    file_path="question_5.txt"
)

In [10]:
write_file(
    question="I booked the wrong dates / times",
    answer="""
If you have found that you have booked the wrong dates or times, please contact the airline or travel agent that you booked your flight with as they will be able to help you change your flights to the intended dates or times.

The search box below can help you find the contact details for the travel provider you booked with.

You can search flexible or specific dates on Skyscanner to find your preferred flight, and when you select a flight on Skyscanner you are transferred to the website where you will make and pay for your booking. Once you are redirected to the airline or travel agent website, you might be required to select dates again, depending on the website. In all cases, you will be shown the flight details of your selection and you are required before confirming payment to state that you have checked all details and agreed to the terms and conditions. We strongly recommend that you always check this information carefully, as travel information can be subject to change.
""".strip(),
    file_path="question_6.txt",
)

In [11]:
write_file(
    question="I entered the wrong email address",
    answer="""
If you made a mistake with your email address when booking, you'll need to get in touch with the company you bought travel from as they'll be able to help.

There are 1000s of travel agencies, airlines, hotels and car hire companies that you can buy from through our site and app. We call them "partners". When you buy from them, they will take your payment (you'll see their name on your bank or credit card statement), contact you to confirm your booking, and provide any help or support you might need.
""".strip(),
    file_path="question_7.txt",
)

In [12]:
write_file(
    question="Luggage",
    answer="""
Depending on the flight provider, the rules, conditions and prices for luggage (including sports equipment) do vary.
It’s always a good idea to check with the airline or travel agent directly (and you should be shown the options when you make your booking).
""".strip(),
    file_path="question_8.txt",
)

In [13]:
write_file(
    question="Changes, cancellations, and refunds",
    answer="""
For any questions about changes, cancellations, and refunds – as well as all other questions about bookings – you'll need to contact the company you bought travel from. They'll have all the info about your booking and can advise you.

You'll find 1000s of travel agencies, airlines, hotels and car hire companies that you can buy from through our site and app. When you buy from one of these travel partners, they will take your payment (you'll see their name on your bank or credit card statement), contact you to confirm your booking, and provide any help or support you might need.
""".strip(),
    file_path="question_9.txt",
)

In [14]:
write_file(
    question="Does Skyscanner charge commission?",
    answer="""
Skyscanner does the hard work for you, and it's free to search for your travel options on our site. We search over 1,200 travel providers to find you the best deal for your flight, hotel or car hire.

See a price you like? When we refer you to book, they pay us, a small fee. And that's all there is to it!
""".strip(),
    file_path="question_11.txt",
)

In [15]:
write_file(
    question="Are my details safe?",
    answer="""
Safeguarding privacy is embedded in our culture and we use a combination of industry-standard methods to protect data.  We’ve strong organisation-wide security programmes, using a range of technical, organisational and administrative security measures and best-practice techniques, depending on the type of data being processed.
""".strip(),
    file_path="question_12.txt",
)

# **Model**

In [16]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [17]:
DEVICE

'cuda:0'

In [18]:
# !git --version

In [19]:
# !git lfs install

In [20]:
# !git clone https://huggingface.co/TheBloke/Nous-Hermes-13B-GPTQ

In [25]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    device=DEVICE)

genertaion_config = GenerationConfig.from_pretrained(model_name_or_path)

In [32]:
question = (
    "Which programming language is more suitable for a beginner?"
)
prompt = f"""
### Instruction: {question}
### Response:
""".strip()

In [33]:
print(prompt)

### Instruction: Which programming language is more suitable for a beginner?
### Response:


In [34]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

CPU times: user 2.15 s, sys: 161 ms, total: 2.31 s
Wall time: 4.19 s


In [35]:
print(tokenizer.decode(output[0]))

<s> ### Instruction: Which programming language is more suitable for a beginner?
### Response:Python is generally considered to be a more beginner-friendly programming language compared to Java.</s>
